In [2]:
# Necesary imports
# Pipeline usage learnt from https://towardsdatascience.com/nlp-with-pipeline-gridsearch-5922266e82f4
!pip install tweet-preprocessor
!pip install icecream
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import preprocessor as p
from sklearn.model_selection import GridSearchCV
from icecream import ic
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler 
from sklearn.datasets import make_classification

In [3]:
# Loading Data for Modeling and Blind Testing

bbc_data = pd.read_csv("bbc_news.csv")
large_data = pd.read_csv("large_news_data.csv")
news_api_data  = pd.read_csv("large_news_data.csv")
test_tweets = pd.read_csv("labeled_tweets.csv") 

In [4]:
# Clean BBC Headlines Data

bbc_data.text = bbc_data.text.str.lower().apply(p.clean)
bbc_data.dropna(inplace=True)
bbc_data

,text,label
0,worldcom ex-boss launches defence lawyers defe...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,enron bosses in $168m payout eighteen former e...,business
4,howard truanted to play snooker conservative l...,politics
...,...,...
1259,great opinion piece from in on the consequence...,health
1260,there are still too many families who are just...,health
1261,this is the last gdp report from trumps tenure...,health
1262,"paid time off to get vaccinated, featuring and...",health


In [5]:
# Clean large headlines data (subset of) : https://www.kaggle.com/rmisra/news-category-dataset 

large_data.dropna(inplace=True)
large_data.text = large_data.text.str.lower()
large_data.text = large_data.text.apply(p.clean)
large_data

,text,label
0,why overeating doesn't make you fat,health
1,habits of people with a healthy relationship t...,health
2,things that could be stealing your joy,health
3,moments make a life,health
4,fat facts,health
...,...,...
24879,vanessa and nick lachey welcome christmas eve ...,entertainment
24880,remembering george michael with of his greates...,entertainment
24881,the first 'alien: covenant' trailer is a terri...,entertainment
24882,'rogue one' dominates the holiday box office,entertainment


In [6]:
# news_api_data.text = news_api_data.text.str.lower()
# news_api_data.text = news_api_data.text.apply(p.clean)
# news_api_data.dropna(inplace=True)
# news_api_data

In [7]:
# Manually annotates test tweets

test_tweets.fillna("NA", inplace=True)
test_tweets.text = test_tweets.text.str.lower()
test_tweets.text = test_tweets.text.apply(p.clean)
test_tweets

,text,label1,label2
0,while athletes and other attendees at this sum...,sports,health
1,the family of a military veteran who died in a...,politics,NA
2,a state lawmaker in missouri was charged this ...,health,politics
3,u.s. defense secretary lloyd austin convened t...,politics,NA
4,who knew that it could be so much fun to watch...,politics,entertainment
...,...,...,...
540,elon musks spacex announces a spaceflight inte...,business,NA
541,"as house gop faces decision on its future, mcc...",politics,NA
542,the gamestop stock situation isnt about populi...,business,NA
543,"opinion: confederate names are coming down, bu...",politics,NA


In [8]:
# Combining bbc data and news category data

combined_bbc_large = pd.concat([bbc_data, large_data])
combined_bbc_large.reset_index(inplace=True)
combined_bbc_large.drop(columns=['index'], inplace=True)
combined_bbc_large

,text,label
0,worldcom ex-boss launches defence lawyers defe...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,enron bosses in $168m payout eighteen former e...,business
4,howard truanted to play snooker conservative l...,politics
...,...,...
26142,vanessa and nick lachey welcome christmas eve ...,entertainment
26143,remembering george michael with of his greates...,entertainment
26144,the first 'alien: covenant' trailer is a terri...,entertainment
26145,'rogue one' dominates the holiday box office,entertainment


In [9]:
training_data = large_data # Set the dataframe you want to train with (options: large_data or combined_bbc_large)

In [10]:
# Create Training/testing Splits and Factorized labels

training_data['category'] = training_data.label.factorize()[0]  # Factorize Labels
X,y = training_data['text'], training_data['category'] 
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0)

In [11]:
# Setting Up Pipelines and Parameters
mnb_pipe = Pipeline([
    ('tfv', TfidfVectorizer()),  
    ('mb', MultinomialNB())
])

log_pipe = Pipeline([
    ('tfv', TfidfVectorizer()),
    ('lm', LogisticRegression())
])

lsvc_pipe = Pipeline([
    ('tfv', TfidfVectorizer()),
    ('linearsvc', LinearSVC(random_state=0))                   

])

mnb_params = {
 'tfv__max_features':[1000, 2000, 3000, 4000, 5000, 6000, 7000, None],
 'tfv__ngram_range': [(1, 1), (1, 2), (2, 2)],
 'tfv__stop_words': ['english']
}

log_params = {
 'tfv__max_features':[1000, 2000, 3000, 4000, 5000, 6000, 7000, None],
 'tfv__ngram_range': [(1, 1), (1, 2), (2, 2)],
 'tfv__stop_words': ['english'],
 'lm__random_state': [0],
 'lm__solver' : ['saga']
}

lsvc_params = {
 'tfv__max_features':[1000, 2000, 3000, 4000, 5000, 6000, 7000, None],
 'tfv__ngram_range': [(1, 1), (1, 2), (2, 2)],
 'tfv__stop_words': ['english']
}


In [12]:
# Linear SVC Grid Search
lsvc_gs = GridSearchCV(lsvc_pipe, param_grid=lsvc_params, cv=5, verbose=1, n_jobs=-1)
lsvc_gs.fit(X_train, y_train)
ic(lsvc_gs.score(X_train, y_train))  # Train Accuracy
ic(lsvc_gs.score(X_test, y_test))    # Test Accuracy

Fitting 5 folds for each of 21 candidates, totalling 105 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   24.9s
[Parallel(n_jobs=-1)]: Done 105 out of 105 | elapsed:   56.5s finished
ic| lsvc_gs.score(X_train, y_train): 0.9995713214017791
ic| lsvc_gs.score(X_test, y_test): 0.8291271499758881


0.8291271499758881

In [13]:
# Multinomial Naive Bayes Grid Search

mnb_gs = GridSearchCV(mnb_pipe, param_grid=mnb_params, cv = 5, verbose =1, n_jobs = -1)
mnb_gs.fit(X_train, y_train)
ic(mnb_gs.score(X_train, y_train)) # Train Acc
ic(mnb_gs.score(X_test, y_test))  # Test Acc

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 21 candidates, totalling 105 fits


[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   17.1s
[Parallel(n_jobs=-1)]: Done 105 out of 105 | elapsed:   39.8s finished
ic| mnb_gs.score(X_train, y_train): 0.9729396634873004
ic| mnb_gs.score(X_test, y_test): 0.8209291110753898


0.8209291110753898

In [14]:
# Logistic Regression Classifier Grid Search 

log_gs = GridSearchCV(log_pipe, param_grid=log_params, cv = 5, verbose =1, n_jobs = -1)
log_gs.fit(X_train, y_train)
ic(log_gs.score(X_train, y_train)) # Train Acc
ic(log_gs.score(X_test, y_test))   # Test Acc

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 21 candidates, totalling 105 fits


[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   25.1s
[Parallel(n_jobs=-1)]: Done 105 out of 105 | elapsed:  1.0min finished
ic| log_gs.score(X_train, y_train): 0.9636694888007716
ic| log_gs.score(X_test, y_test): 0.8130525638964796


0.8130525638964796

In [15]:
training_data[['category', 'label']].drop_duplicates()


,category,label
0,0,health
5000,1,business
10000,2,politics
15000,3,sports
19884,4,entertainment


In [16]:
test_news = test_tweets # Setting which data to test on

In [17]:
test_news.text = test_news.text.str.lower().apply(p.clean)
test_news.dropna(inplace=True)

In [18]:
test_news

,text,label1,label2
0,while athletes and other attendees at this sum...,sports,health
1,the family of a military veteran who died in a...,politics,NA
2,a state lawmaker in missouri was charged this ...,health,politics
3,u.s. defense secretary lloyd austin convened t...,politics,NA
4,who knew that it could be so much fun to watch...,politics,entertainment
...,...,...,...
540,elon musks spacex announces a spaceflight inte...,business,NA
541,"as house gop faces decision on its future, mcc...",politics,NA
542,the gamestop stock situation isnt about populi...,business,NA
543,"opinion: confederate names are coming down, bu...",politics,NA


In [19]:
# Returns label, given a numerical factor associated with it

def return_label(key):
    # labels = {0: "business", 1: "politics", 2: "sports", 3: "entertainment", 4: "health"} # Use with combined_bbc_large
    labels = {0: "health", 1: "business", 2: "politics", 3: "sports", 4: "entertainment"} # Use with large_data
    return labels[key]

In [20]:
# Predictions (appended to the testing data as columns)

y_svc = lsvc_gs.predict(test_news.text)
y_mnb = mnb_gs.predict(test_news.text)
y_log = log_gs.predict(test_news.text)

y_svc = list(map(return_label, y_svc))
y_mnb = list(map(return_label, y_mnb))
y_log = list(map(return_label, y_log))

test_news['y_svc'] = y_svc  # SVC Predictions
test_news['y_mnb'] = y_mnb  # MNB Predictions
test_news['y_log'] = y_log  # LOG Predictions

In [21]:
test_news

,text,label1,label2,y_svc,y_mnb,y_log
0,while athletes and other attendees at this sum...,sports,health,sports,sports,sports
1,the family of a military veteran who died in a...,politics,NA,politics,politics,politics
2,a state lawmaker in missouri was charged this ...,health,politics,health,politics,health
3,u.s. defense secretary lloyd austin convened t...,politics,NA,politics,politics,politics
4,who knew that it could be so much fun to watch...,politics,entertainment,sports,entertainment,sports
...,...,...,...,...,...,...
540,elon musks spacex announces a spaceflight inte...,business,NA,business,business,business
541,"as house gop faces decision on its future, mcc...",politics,NA,politics,politics,politics
542,the gamestop stock situation isnt about populi...,business,NA,business,business,business
543,"opinion: confederate names are coming down, bu...",politics,NA,politics,politics,politics


In [22]:
# Comparing labels across columns to check for accuracy

test_news['svc_preds'] = (test_news['y_svc'] == test_news.label1) | (test_news['y_svc'] == test_news.label2)
test_news['mnb_preds'] = (test_news['y_mnb'] == test_news.label1) | (test_news['y_mnb'] == test_news.label2)
test_news['log_preds'] = (test_news['y_log'] == test_news.label1) | (test_news['y_log'] == test_news.label2)

In [23]:
test_news

,text,label1,label2,y_svc,y_mnb,y_log,svc_preds,mnb_preds,log_preds
0,while athletes and other attendees at this sum...,sports,health,sports,sports,sports,True,True,True
1,the family of a military veteran who died in a...,politics,NA,politics,politics,politics,True,True,True
2,a state lawmaker in missouri was charged this ...,health,politics,health,politics,health,True,True,True
3,u.s. defense secretary lloyd austin convened t...,politics,NA,politics,politics,politics,True,True,True
4,who knew that it could be so much fun to watch...,politics,entertainment,sports,entertainment,sports,False,True,False
...,...,...,...,...,...,...,...,...,...
540,elon musks spacex announces a spaceflight inte...,business,NA,business,business,business,True,True,True
541,"as house gop faces decision on its future, mcc...",politics,NA,politics,politics,politics,True,True,True
542,the gamestop stock situation isnt about populi...,business,NA,business,business,business,True,True,True
543,"opinion: confederate names are coming down, bu...",politics,NA,politics,politics,politics,True,True,True


In [24]:
svc_accuracy = test_news.svc_preds.value_counts()[True]/len(test_news)
mnb_accuracy = test_news.mnb_preds.value_counts()[True]/len(test_news)
log_accuracy = test_news.log_preds.value_counts()[True]/len(test_news)

ic(svc_accuracy)  # SVC Acc
ic(mnb_accuracy)  # MNB Acc
ic(log_accuracy)  # Log Acc

ic| svc_accuracy: 0.7963302752293578
ic| mnb_accuracy: 0.8146788990825689
ic| log_accuracy: 0.781651376146789


0.781651376146789

In [ ]:
# ACCURACIES ON LARGE DATA (NEWS CATEGORY DATASET)

# ic| svc_accuracy: 0.7963302752293578
# ic| mnb_accuracy: 0.8146788990825689
# ic| log_accuracy: 0.781651376146789
# 0.781651376146789

In [25]:
# ACCURACIES ON COMBINED DATA


# ic| svc_accuracy: 0.8036697247706422
# ic| mnb_accuracy: 0.8165137614678899
# ic| log_accuracy: 0.7871559633027523
# 0.7871559633027523

In [26]:
lsvc_gs.best_estimator_

Pipeline(memory=None,
         steps=[('tfv',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words='english', strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('linearsvc',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
      

In [27]:
mnb_gs.best_estimator_

Pipeline(memory=None,
         steps=[('tfv',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words='english', strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('mb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [28]:
log_gs.best_estimator_

Pipeline(memory=None,
         steps=[('tfv',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words='english', strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('lm',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scali